# Developing "Momentum" Trading Strategy
* Many differnt takes on this strategy
* Sometimes referred to as trend following
* Whatever you do and call it, you first need to research potential trading signals

In [1]:
import numpy as np
import pandas as pd
import pandas_datareader as pdr
import matplotlib.pyplot as plt


##### Import Data

### 1. Download data: 
* Many services for this, some paid some free 
* Yahoo Finance API
* Typically trading "systems" involve a number of securities
* For this demonstration we are just going to look at GLD --> the gold ETF

In [2]:
gld = pdr.get_data_yahoo('GLD')
day = np.arange(1, len(gld) + 1)
gld['day'] = day
gld.drop(columns=['Adj Close', 'Volume'], inplace = True)
gld = gld[['day', 'Open', 'High', 'Low', 'Close']]
gld.head()

,day,Open,High,Low,Close
Date,,,,,
2016-06-23,1,120.489998,121.029999,120.059998,120.110001
2016-06-24,2,126.620003,126.820000,125.000000,126.000000
2016-06-27,3,126.440002,127.050003,125.830002,126.680000
2016-06-28,4,125.559998,126.019997,125.059998,125.320000
2016-06-29,5,125.910004,126.809998,125.699997,125.839996


In [3]:
gld.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1258 entries, 2016-06-23 to 2021-06-22
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   day     1258 non-null   int32  
 1   Open    1258 non-null   float64
 2   High    1258 non-null   float64
 3   Low     1258 non-null   float64
 4   Close   1258 non-null   float64
dtypes: float64(4), int32(1)
memory usage: 54.1 KB


### 2. Add data/transform data
* calculate signal based on some price or statistical action
* we are going to try a moving average crossover to generate signals
* for this strategy we will always by "in" a trade, either long or short
* we are modeling; this means real life variation should be expected

#### Add moving averages to the data frame

In [4]:
gld['9-day'] = gld['Close'].rolling(9).mean()
gld['21-day'] = gld['Close'].rolling(21).mean()
gld[19:25]

,day,Open,High,Low,Close,9-day,21-day
Date,,,,,,,
2016-07-21,20,125.660004,127.370003,125.599998,127.300003,127.318889,NaN
2016-07-22,21,126.430000,126.519997,126.010002,126.349998,126.992223,127.190953
2016-07-25,22,125.529999,126.209999,125.110001,125.470001,126.805556,127.446191
2016-07-26,23,126.089996,126.139999,125.750000,126.000000,126.547778,127.446191
2016-07-27,24,126.790001,128.119995,126.040001,128.029999,126.625555,127.510477
2016-07-28,25,128.089996,128.199997,127.209999,127.660004,126.716667,127.621905


#### Add "signal" column  

In [5]:
gld['signal'] = np.where(gld['9-day'] > gld['21-day'], 1, 0)
gld['signal'] = np.where(gld['9-day'] < gld['21-day'], -1, gld['signal'])
gld.dropna(inplace=True)
gld.head()

,day,Open,High,Low,Close,9-day,21-day,signal
Date,,,,,,,,
2016-07-22,21,126.430000,126.519997,126.010002,126.349998,126.992223,127.190953,-1
2016-07-25,22,125.529999,126.209999,125.110001,125.470001,126.805556,127.446191,-1
2016-07-26,23,126.089996,126.139999,125.750000,126.000000,126.547778,127.446191,-1
2016-07-27,24,126.790001,128.119995,126.040001,128.029999,126.625555,127.510477,-1
2016-07-28,25,128.089996,128.199997,127.209999,127.660004,126.716667,127.621905,-1


#### Calculate Instantaneous returns/system returns

In [6]:
gld['return'] = np.log(gld['Close']).diff()
gld['system_return'] = gld['signal'] * gld['return']
gld['entry'] = gld.signal.diff()
gld.head()

,day,Open,High,Low,Close,9-day,21-day,signal,return,system_return,entry
Date,,,,,,,,,,,
2016-07-22,21,126.430000,126.519997,126.010002,126.349998,126.992223,127.190953,-1,NaN,NaN,NaN
2016-07-25,22,125.529999,126.209999,125.110001,125.470001,126.805556,127.446191,-1,-0.006989,0.006989,0.0
2016-07-26,23,126.089996,126.139999,125.750000,126.000000,126.547778,127.446191,-1,0.004215,-0.004215,0.0
2016-07-27,24,126.790001,128.119995,126.040001,128.029999,126.625555,127.510477,-1,0.015983,-0.015983,0.0
2016-07-28,25,128.089996,128.199997,127.209999,127.660004,126.716667,127.621905,-1,-0.002894,0.002894,0.0


#### Plot trades on time series

In [7]:
plt.rcParams['figure.figsize'] = 12, 6
plt.grid(True, alpha = .3)
plt.plot(gld.iloc[-252:]['Close'], label = 'GLD')
plt.plot(gld.iloc[-252:]['9-day'], label = '9-day')
plt.plot(gld.iloc[-252:]['21-day'], label = '21-day')
plt.plot(gld[-252:].loc[gld.entry == 2].index, gld[-252:]['9-day'][gld.entry == 2], '^',
         color = 'g', markersize = 12)
plt.plot(gld[-252:].loc[gld.entry == -2].index, gld[-252:]['21-day'][gld.entry == -2], 'v',
         color = 'r', markersize = 12)
plt.legend(loc=2);

In [8]:
plt.plot(np.exp(gld['return']).cumprod(), label='Buy/Hold')
plt.plot(np.exp(gld['system_return']).cumprod(), label='System')
plt.legend(loc=2)
plt.grid(True, alpha=.3)

In [9]:
np.exp(gld['return']).cumprod()[-1] -1

0.3157894895744784

In [10]:
np.exp(gld['system_return']).cumprod()[-1] -1

0.8175253684655783